<a href="https://colab.research.google.com/github/AnoshkinDV/Analysis-Data/blob/master/task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:
InvoiceNo — номер транзакции
StockCode — код товара
Description — описание товара
Quantity — количество единиц товара, добавленных в заказ
InvoiceDate — дата транзакции
UnitPrice — цена за единицу товара
CustomerID — id клиента
Country — страна, где проживает клиент

In [1]:
import pandas as pd


In [2]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/data.csv.zip'

In [59]:
retail = pd.read_csv(path_to_file,encoding='ISO-8859-1',compression='zip')

In [60]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [61]:
retail_columns = retail.columns

In [62]:
retail.shape

(541909, 8)

In [63]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [64]:
# Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.
retail[retail.duplicated()].InvoiceNo.count()

5268

In [65]:
retail = retail.drop_duplicates()

In [66]:
# Данные содержат в себе записи как об успешных транзакциях, так и об отмененных.
# Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).
# Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакция.

#Как выделить первый символ из строки каждой колонки?
retail.InvoiceNo.str[0]

,InvoiceNo
0,5
1,5
2,5
3,5
4,5
...,...
541904,5
541905,5
541906,5
541907,5


In [67]:
retail.InvoiceNo.str[0].unique() #Какие бывают уникальные префиксы у номеров транзакции, он может начинаться с  ['5', 'C', 'A']

array(['5', 'C', 'A'], dtype=object)

In [68]:
retail[retail.InvoiceNo.str[0] == 'C'].shape#Ответ будет 9251 отменённых транзакции

(9251, 8)

In [69]:
#Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.
# retail.loc[retail.Quantity > 0]
retail.query("Quantity > 0").shape

(526054, 8)

In [70]:
retail = retail.loc[retail.Quantity > 0]

In [ ]:
# Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany).
# Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.
# Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).
#Нужно учесть, что  для каждого заказа в данных может встречаться более 1 строки.

In [87]:
N = retail.query("Country == 'Germany'").groupby('CustomerID',as_index=False)\
                                    .agg({'InvoiceNo':'nunique'})\
                                    .InvoiceNo.quantile(0.8)
#Нужно посчитать количество совершенных заказов, при этом для каждого заказа может быть больше одной строки в данных, поэтому считаем nunique

In [90]:
germany_top = retail.query("Country == 'Germany'").groupby('CustomerID',as_index=False)\
                                    .agg({'InvoiceNo':'nunique'})\
                                    .query('InvoiceNo > @N') \
                                    .CustomerID

In [92]:
N

7.0

In [91]:
germany_top

,CustomerID
3,12471.0
6,12474.0
8,12476.0
12,12481.0
16,12500.0
26,12524.0
37,12569.0
44,12600.0
51,12619.0
52,12621.0


In [ ]:
# Используя объект с id пользователей (germany_top), полученный на предыдущем шаге,
# отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм
# запишите в top_retail_germany.
# Датафрейм записан в retail, нужные id пользователей – в germany_top

In [97]:
# retail.loc[retail.CustomerID.isin(germany_top)]
top_retail_germany = retail.query('CustomerID in @germany_top')

In [ ]:
# Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?
# Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [114]:
top_retail_germany.query("StockCode != 'POST'").groupby('StockCode',as_index=False)\
                                               .agg({'InvoiceNo':'count'})\
                                               .sort_values('InvoiceNo',ascending=False)

,StockCode,InvoiceNo
409,22326,52
411,22328,38
453,22423,34
45,20719,30
709,22961,28
...,...,...
350,22168,1
769,23104,1
768,23102,1
354,22174,1


In [ ]:
# Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.
# Отфильтрованные данные записаны в retail.

In [119]:
del retail['sum']

In [123]:
# retail['Revenue'] = retail.Quantity * retail.UnitPrice
retail['Revenue'] = retail.Quantity.mul(retail.UnitPrice)

In [125]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [137]:
# Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.
# В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)
top = retail.groupby('InvoiceNo',as_index=False)\
      .agg({'Revenue':'sum'})\
      .rename(columns= {'Revenue':'TotalRevenue'})\
      .sort_values('TotalRevenue',ascending=False)\
      .head()

In [140]:
top

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00


In [138]:
top_5 = [i for i in top.InvoiceNo]

In [139]:
top_5

['581483', '541431', '574941', '576365', '556444']